# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
from pprint import pprint

# Import API key
from config_google import g_key

In [2]:
path = "weather_data_output.csv"

# Read the data 
weather_data = pd.read_csv(path)
weather_data

,Country,City,Latitude (Degrees),Longitude (Degrees),Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,TO,vaini,-21.2000,-175.2000,84.20,79,75,4.61
1,AU,esperance,-33.8667,121.9000,91.40,21,16,11.50
2,CL,punta arenas,-53.1500,-70.9167,50.00,66,90,20.71
3,RU,saskylakh,71.9167,114.0833,-20.72,85,100,14.14
4,RU,makaryev,57.8797,43.8069,-7.22,90,85,9.01
...,...,...,...,...,...,...,...,...
552,IN,mangrol,21.1167,70.1167,75.36,58,0,4.59
553,MZ,maxixe,-23.8597,35.3472,77.00,100,75,10.40
554,CI,tabou,4.4230,-7.3528,77.97,90,38,6.33
555,PE,barranca,-10.7500,-77.7667,69.64,83,0,3.98


In [3]:
weather_data["Humidity (%)"].max()

100

### Humidity Heatmap

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = weather_data[["Latitude (Degrees)", "Longitude (Degrees)"]]
Humidity = weather_data["Humidity (%)"]

fig = gmaps.figure()

# Create heat layer; ask HUNTER about adjusting max_intensity (note on line 25 of readme)
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=Humidity, 
    dissipating=False, 
    max_intensity=5,
    point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Ideal weather: 70 - 85 degrees Fahrenheit
temp = (weather_data["Temperature (F)"] > 75) & (weather_data["Temperature (F)"]<=85)
wind = weather_data["Wind Speed (mph)"]<=10.00
cloud = weather_data["Cloudiness (%)"]==0

ideal_weather_df = weather_data[temp & wind & cloud]
ideal_weather_df

,Country,City,Latitude (Degrees),Longitude (Degrees),Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
27,HK,victoria,22.2855,114.1577,75.99,44,0,1.99
94,OM,sur,22.5667,59.5289,76.10,58,0,9.46
158,IN,saiha,22.4833,92.9667,83.34,14,0,4.70
373,LA,luang prabang,19.8856,102.1347,80.60,47,0,2.30
415,SA,umm lajj,25.0213,37.2685,75.70,57,0,3.40
447,PK,khanpur,28.6453,70.6567,82.81,14,0,1.97
511,IN,bikaner,28.0167,73.3000,81.97,17,0,7.76
518,OM,nizwa,22.9333,57.5333,78.33,21,0,3.47
552,IN,mangrol,21.1167,70.1167,75.36,58,0,4.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Add a new column for hotel name
ideal_weather_df["Hotel Name"]=" "
ideal_weather_df

<ipython-input-7-ee732607200c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel Name"]=" "


,Country,City,Latitude (Degrees),Longitude (Degrees),Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
27,HK,victoria,22.2855,114.1577,75.99,44,0,1.99,
94,OM,sur,22.5667,59.5289,76.10,58,0,9.46,
158,IN,saiha,22.4833,92.9667,83.34,14,0,4.70,
373,LA,luang prabang,19.8856,102.1347,80.60,47,0,2.30,
415,SA,umm lajj,25.0213,37.2685,75.70,57,0,3.40,
447,PK,khanpur,28.6453,70.6567,82.81,14,0,1.97,
511,IN,bikaner,28.0167,73.3000,81.97,17,0,7.76,
518,OM,nizwa,22.9333,57.5333,78.33,21,0,3.47,
552,IN,mangrol,21.1167,70.1167,75.36,58,0,4.59,


In [15]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotels",
    "key": g_key
}

name_hotel = []

for index, row in ideal_weather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude (Degrees)"]
    lng = row["Longitude (Degrees)"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    
    hotel = hotel.json()
    print(json.dumps(hotel, indent=4, sort_keys=True))
    #pprint(hotel)
    
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        name_hotel.append(hotel['results'][3]['name'])
    except IndexError:
        name_hotel.append(np.nan)


In [16]:
name_hotel
# name_hotel.append(hotel['results'][0]['geometry'])
# name_hotel

['The Harbourview',
 'Sur Bandar Ayjah Hotel Apartments صور بندر العيجة للشقق الفندقية',
 nan,
 'Victoria Xiengthong Palace',
 'Hotel New Furnished Family Rooms',
 'ALLAH Waris Hotel',
 'Hotel Bhanwar Niwas',
 'Orient Apartments',
 nan]

In [22]:
hotel['results'][3]['name']

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_fig = gmaps.figure(center=[34.00510025,-118.4349976], zoom_level=11)

marker_layer = gmaps.marker_layer(grouped_reg_data[['Regular Lat', 'Regular Long']],
                                hover_text= hover_reg,
                                label=grouped_reg_data.index,
                                info_box_content=None,
                                display_info_box=None)

reg_fig.add_layer(marker_layer)

# Display figure
